In [ ]:
# 1회 이상 실행시 오류가 발생합니다.
! mkdir -p /home/faith/workspace/cifar/generated_samples
! mkdir -p /home/faith/workspace/cifar/training_checkpoints
! mkdir -p /home/faith/workspace/cifar/training_history

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import time
import os

# 저장 경로 설정
SAVE_DIR = './cgan_results'
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(os.path.join(SAVE_DIR, 'images'), exist_ok=True)
os.makedirs(os.path.join(SAVE_DIR, 'models'), exist_ok=True)

# 데이터 로드 및 전처리
(train_images, train_labels), (_, _) = tf.keras.datasets.cifar10.load_data()
train_images = (train_images.astype('float32') - 127.5) / 127.5  # 이미지를 [-1, 1]로 정규화

BUFFER_SIZE = 60000
BATCH_SIZE = 256

# 데이터셋 준비
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# 생성자 모델 정의
def make_generator_model():
    noise_input = layers.Input(shape=(100,))
    label_input = layers.Input(shape=(10,))
    
    x = layers.Concatenate()([noise_input, label_input])
    x = layers.Dense(8*8*256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Reshape((8, 8, 256))(x)
    
    x = layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')(x)

    model = tf.keras.Model([noise_input, label_input], x)
    return model

# 판별자 모델 정의
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[32, 32, 13]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

# 손실 함수 정의
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

# 모델 생성
generator = make_generator_model()
discriminator = make_discriminator_model()

# 옵티마이저 초기화
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)
@tf.function
def train_step(images, labels):
    batch_size = tf.shape(images)[0]
    noise = tf.random.normal([batch_size, 100])
    
    # 이미지를 float32로 변환
    images = tf.cast(images, tf.float32)
    
    # 원-핫 인코딩된 레이블 생성
    one_hot_labels = tf.one_hot(tf.squeeze(labels), depth=10)

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator([noise, one_hot_labels], training=True)

        # 레이블을 이미지와 결합
        label_channels = tf.repeat(tf.expand_dims(one_hot_labels, axis=1), repeats=32, axis=1)
        label_channels = tf.repeat(tf.expand_dims(label_channels, axis=2), repeats=32, axis=2)
        
        real_input = tf.concat([images, label_channels], axis=3)
        fake_input = tf.concat([generated_images, label_channels], axis=3)

        real_output = discriminator(real_input, training=True)
        fake_output = discriminator(fake_input, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

def generate_and_save_images(model, epoch, test_input, test_labels):
    predictions = model([test_input, test_labels], training=False)

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow((predictions[i, :, :, :] + 1) / 2.0)
        plt.axis('off')
        plt.title(f'Class {tf.argmax(test_labels[i])}')

    plt.savefig(os.path.join(SAVE_DIR, 'images', f'image_at_epoch_{epoch:04d}.png'))
    plt.close()

def train(dataset, epochs):
    history = []
    
    # 테스트용 입력 생성
    num_examples_to_generate = 16
    test_noise = tf.random.normal([num_examples_to_generate, 100])
    test_labels = tf.range(0, 10, delta=1)
    test_labels = tf.concat([test_labels, tf.random.uniform([6], minval=0, maxval=10, dtype=tf.int32)], axis=0)
    test_labels = tf.one_hot(test_labels, depth=10)
    
    for epoch in range(epochs):
        start = time.time()
        
        epoch_gen_loss = []
        epoch_disc_loss = []

        for image_batch, label_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch, label_batch)
            epoch_gen_loss.append(gen_loss)
            epoch_disc_loss.append(disc_loss)

        # 에포크 평균 손실 계산
        avg_gen_loss = tf.reduce_mean(epoch_gen_loss)
        avg_disc_loss = tf.reduce_mean(epoch_disc_loss)
        
        history.append({
            'epoch': epoch + 1,
            'gen_loss': avg_gen_loss.numpy(),
            'disc_loss': avg_disc_loss.numpy(),
            'time': time.time() - start
        })

        print(f'Epoch {epoch + 1}, Gen Loss: {avg_gen_loss:.4f}, Disc Loss: {avg_disc_loss:.4f}, Time: {time.time()-start:.2f} sec')

        # 에포크마다 이미지 생성 및 저장
        generate_and_save_images(generator, epoch + 1, test_noise, test_labels)
        
        # 10 에포크마다 모델 저장
        if (epoch + 1) % 10 == 0:
            generator.save(os.path.join(SAVE_DIR, 'models', f'generator_epoch_{epoch+1}.h5'))
            discriminator.save(os.path.join(SAVE_DIR, 'models', f'discriminator_epoch_{epoch+1}.h5'))

        display.clear_output(wait=True)
        generate_and_save_images(generator, epochs, it, seed)
    
    # 학습 히스토리 저장
    np.save(os.path.join(SAVE_DIR, 'training_history.npy'), history)
    
    # 학습 히스토리 그래프 생성 및 저장
    plot_history(history)

def plot_history(history):
    gen_loss = [h['gen_loss'] for h in history]
    disc_loss = [h['disc_loss'] for h in history]
    epochs = [h['epoch'] for h in history]

    plt.figure(figsize=(10, 5))
    plt.plot(epochs, gen_loss, label='Generator Loss')
    plt.plot(epochs, disc_loss, label='Discriminator Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training History')
    plt.savefig(os.path.join(SAVE_DIR, 'training_history.png'))
    plt.close()

# 모델 학습
EPOCHS = 50
train(train_dataset, EPOCHS)

# 학습된 모델을 사용하여 특정 클래스의 이미지 생성
def generate_specific_class(class_num):
    noise = tf.random.normal([1, 100])
    one_hot_label = tf.one_hot([class_num], depth=10)
    generated_image = generator([noise, one_hot_label], training=False)
    
    plt.figure(figsize=(4, 4))
    plt.imshow((generated_image[0, :, :, :] + 1) / 2.0)
    plt.axis('off')
    plt.title(f'Generated image for class {class_num}')
    plt.savefig(os.path.join(SAVE_DIR, f'generated_class_{class_num}.png'))
    plt.close()

# 클래스 0부터 9까지의 이미지 생성
for i in range(10):
    generate_specific_class(i)

2024-08-21 14:47:42.260850: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 14:47:42.274130: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 14:47:42.277806: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-21 14:47:42.287674: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-21 14:47:43.056520: W tensorflow/compiler/tf2

Epoch 1, Gen Loss: 1.0001, Disc Loss: 1.1038, Time: 14.04 sec


W0000 00:00:1724219281.522748   30730 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724219281.524160   30730 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724219281.525327   30730 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724219281.527695   30730 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724219281.528949   30730 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724219281.530599   30730 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724219281.532326   30730 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724219281.541167   30730 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724219281.542054   30730 gp

Epoch 2, Gen Loss: 1.5230, Disc Loss: 0.8244, Time: 8.73 sec
Epoch 3, Gen Loss: 1.8970, Disc Loss: 0.7115, Time: 10.65 sec


2024-08-21 14:48:30.596756: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

Epoch 31, Gen Loss: 1.4000, Disc Loss: 1.0216, Time: 9.55 sec


2024-08-21 14:53:08.573288: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 32, Gen Loss: 1.3122, Disc Loss: 1.0437, Time: 8.59 sec
Epoch 33, Gen Loss: 1.3135, Disc Loss: 1.1166, Time: 8.68 sec
Epoch 34, Gen Loss: 1.4088, Disc Loss: 1.0256, Time: 11.13 sec
Epoch 35, Gen Loss: 1.3790, Disc Loss: 1.0433, Time: 8.70 sec
Epoch 36, Gen Loss: 1.3004, Disc Loss: 1.1020, Time: 8.69 sec
Epoch 37, Gen Loss: 1.4525, Disc Loss: 0.9619, Time: 10.82 sec
Epoch 38, Gen Loss: 1.3548, Disc Loss: 1.0305, Time: 8.76 sec
Epoch 39, Gen Loss: 1.3717, Disc Loss: 1.0448, Time: 8.70 sec
Epoch 40, Gen Loss: 1.6236, Disc Loss: 0.9529, Time: 8.77 sec


Epoch 41, Gen Loss: 1.2827, Disc Loss: 1.0768, Time: 10.84 sec
Epoch 42, Gen Loss: 1.3322, Disc Loss: 1.0749, Time: 8.63 sec
Epoch 43, Gen Loss: 1.2927, Disc Loss: 1.1098, Time: 8.58 sec
Epoch 44, Gen Loss: 1.1380, Disc Loss: 1.1841, Time: 8.71 sec
Epoch 45, Gen Loss: 1.3517, Disc Loss: 0.9857, Time: 10.96 sec
Epoch 46, Gen Loss: 1.2959, Disc Loss: 1.1315, Time: 9.52 sec
Epoch 47, Gen Loss: 1.3205, Disc Loss: 1.0485, Time: 8.70 sec
Epoch 48, Gen Loss: 1.2265, Disc Loss: 1.1134, Time: 10.79 sec
Epoch 49, Gen Loss: 1.2047, Disc Loss: 1.1587, Time: 8.73 sec
Epoch 50, Gen Loss: 1.2365, Disc Loss: 1.2012, Time: 8.72 sec


W0000 00:00:1724219763.218347   30730 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724219763.219201   30730 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724219763.220453   30730 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724219763.221231   30730 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724219763.222240   30730 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724219763.223060   30730 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724219763.224330   30730 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724219763.225945   30730 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724219763.227969   30730 gp

In [5]:
noise_dim = 100
num_examples_to_generate = 16

seed = tf.random.normal([num_examples_to_generate, noise_dim])
seed.shape

TensorShape([16, 100])

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import time
import os

# 저장 경로 설정
SAVE_DIR = './cgan_results'
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(os.path.join(SAVE_DIR, 'images'), exist_ok=True)
os.makedirs(os.path.join(SAVE_DIR, 'models'), exist_ok=True)

# 데이터 로드 및 전처리
(train_images, train_labels), (_, _) = tf.keras.datasets.cifar10.load_data()
train_images = (train_images.astype('float32') - 127.5) / 127.5  # 이미지를 [-1, 1]로 정규화

BUFFER_SIZE = 60000
BATCH_SIZE = 256

# 데이터셋 준비
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# 생성자 모델 정의
def make_generator_model():
    noise_input = layers.Input(shape=(100,))
    label_input = layers.Input(shape=(10,))
    
    x = layers.Concatenate()([noise_input, label_input])
    x = layers.Dense(8*8*256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Reshape((8, 8, 256))(x)
    
    x = layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')(x)

    model = tf.keras.Model([noise_input, label_input], x)
    return model

# 판별자 모델 정의
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[32, 32, 13]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

# 손실 함수 정의
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

# 모델 생성
generator = make_generator_model()
discriminator = make_discriminator_model()

# 옵티마이저 초기화
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)
@tf.function
def train_step(images, labels):
    batch_size = tf.shape(images)[0]
    noise = tf.random.normal([batch_size, 100])
    
    # 이미지를 float32로 변환
    images = tf.cast(images, tf.float32)
    
    # 원-핫 인코딩된 레이블 생성
    one_hot_labels = tf.one_hot(tf.squeeze(labels), depth=10)

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator([noise, one_hot_labels], training=True)

        # 레이블을 이미지와 결합
        label_channels = tf.repeat(tf.expand_dims(one_hot_labels, axis=1), repeats=32, axis=1)
        label_channels = tf.repeat(tf.expand_dims(label_channels, axis=2), repeats=32, axis=2)
        
        real_input = tf.concat([images, label_channels], axis=3)
        fake_input = tf.concat([generated_images, label_channels], axis=3)

        real_output = discriminator(real_input, training=True)
        fake_output = discriminator(fake_input, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss



In [2]:
checkpoint_dir = '/home/faith/workspace/cifar/training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)
                                 

In [3]:

def plot_history(history):
    gen_loss = [h['gen_loss'] for h in history]
    disc_loss = [h['disc_loss'] for h in history]
    epochs = [h['epoch'] for h in history]

    plt.figure(figsize=(10, 5))
    plt.plot(epochs, gen_loss, label='Generator Loss')
    plt.plot(epochs, disc_loss, label='Discriminator Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training History')
    plt.savefig(os.path.join(SAVE_DIR, 'training_history.png'))
    plt.close()

In [4]:

def generate_and_save_images(model, epoch, test_input, test_labels):
    predictions = model([test_input, test_labels], training=False)

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow((predictions[i, :, :, :] + 1) / 2.0)
        plt.axis('off')
        plt.title(f'Class {tf.argmax(test_labels[i])}')

    plt.savefig(os.path.join(SAVE_DIR, 'images', f'image_at_epoch_{epoch:04d}.png'))
    plt.close()

In [6]:
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6    # matlab 차트의 기본 크기를 15,6으로 지정해 줍니다.

def draw_train_history(history, epoch):
    # summarize history for loss  
    plt.subplot(211)  
    plt.plot(history['gen_loss'])  
    plt.plot(history['disc_loss'])  
    plt.title('model loss')  
    plt.ylabel('loss')  
    plt.xlabel('batch iters')  
    plt.legend(['gen_loss', 'disc_loss'], loc='upper left')  

    # summarize history for accuracy  
    plt.subplot(212)  
    plt.plot(history['fake_accuracy'])  
    plt.plot(history['real_accuracy'])  
    plt.title('discriminator accuracy')  
    plt.ylabel('accuracy')  
    plt.xlabel('batch iters')  
    plt.legend(['fake_accuracy', 'real_accuracy'], loc='upper left')  
    
    # training_history 디렉토리에 epoch별로 그래프를 이미지 파일로 저장합니다.
    plt.savefig('/home/faith/workspace/cifar/training_history/train_history_{:04d}.png'
                    .format(epoch))
    plt.show()

In [ ]:

def train(dataset, epochs, save_every):
    history = {'gen_loss':[], 'disc_loss':[], 'real_accuracy':[], 'fake_accuracy':[]}
    
    # 테스트용 입력 생성
    num_examples_to_generate = 16
    test_noise = tf.random.normal([num_examples_to_generate, 100])
    test_labels = tf.range(0, 10, delta=1)
    test_labels = tf.concat([test_labels, tf.random.uniform([6], minval=0, maxval=10, dtype=tf.int32)], axis=0)
    test_labels = tf.one_hot(test_labels, depth=10)
    
    for epoch in range(epochs):
        start = time.time()
        
        epoch_gen_loss = []
        epoch_disc_loss = []

        for image_batch, label_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch, label_batch)
            epoch_gen_loss.append(gen_loss)
            epoch_disc_loss.append(disc_loss)

        # 에포크 평균 손실 계산
        avg_gen_loss = tf.reduce_mean(epoch_gen_loss)
        avg_disc_loss = tf.reduce_mean(epoch_disc_loss)
        
        history.append({
            'epoch': epoch + 1,
            'gen_loss': avg_gen_loss.numpy(),
            'disc_loss': avg_disc_loss.numpy(),
            'time': time.time() - start
        })

        print(f'Epoch {epoch + 1}, Gen Loss: {avg_gen_loss:.4f}, Disc Loss: {avg_disc_loss:.4f}, Time: {time.time()-start:.2f} sec')

        # 에포크마다 이미지 생성 및 저장
        generate_and_save_images(generator, epoch + 1, test_noise, test_labels)
        
        if (epoch + 1) % save_every == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

        display.clear_output(wait=True)
    
    # 학습 히스토리 저장
    np.save(os.path.join(SAVE_DIR, 'training_history.npy'), history)
    
    # 학습 히스토리 그래프 생성 및 저장
    plot_history(history)

def plot_history(history):
    gen_loss = [h['gen_loss'] for h in history]
    disc_loss = [h['disc_loss'] for h in history]
    epochs = [h['epoch'] for h in history]

    plt.figure(figsize=(10, 5))
    plt.plot(epochs, gen_loss, label='Generator Loss')
    plt.plot(epochs, disc_loss, label='Discriminator Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training History')
    plt.savefig(os.path.join(SAVE_DIR, 'training_history.png'))
    plt.close()


# 학습된 모델을 사용하여 특정 클래스의 이미지 생성
def generate_specific_class(class_num):
    noise = tf.random.normal([1, 100])
    one_hot_label = tf.one_hot([class_num], depth=10)
    generated_image = generator([noise, one_hot_label], training=False)
    
    plt.figure(figsize=(4, 4))
    plt.imshow((generated_image[0, :, :, :] + 1) / 2.0)
    plt.axis('off')
    plt.title(f'Generated image for class {class_num}')
    plt.savefig(os.path.join(SAVE_DIR, f'generated_class_{class_num}.png'))
    plt.close()



In [7]:

# 모델 학습
EPOCHS = 50
save_every = 5

train(train_dataset, EPOCHS)

# 클래스 0부터 9까지의 이미지 생성
for i in range(10):
    generate_specific_class(i)

Epoch 1, Gen Loss: 1.4216, Disc Loss: 1.0050, Time: 11.54 sec
Epoch 2, Gen Loss: 1.3224, Disc Loss: 1.1014, Time: 13.81 sec
Epoch 3, Gen Loss: 1.3320, Disc Loss: 1.0300, Time: 10.44 sec
Epoch 4, Gen Loss: 1.3161, Disc Loss: 1.0808, Time: 10.85 sec
Epoch 5, Gen Loss: 1.3832, Disc Loss: 1.0459, Time: 13.62 sec
Epoch 6, Gen Loss: 1.4459, Disc Loss: 1.0321, Time: 8.26 sec
Epoch 7, Gen Loss: 1.3947, Disc Loss: 1.0752, Time: 8.44 sec
Epoch 8, Gen Loss: 1.2744, Disc Loss: 1.0775, Time: 11.45 sec
Epoch 9, Gen Loss: 1.2456, Disc Loss: 1.0985, Time: 8.34 sec
Epoch 10, Gen Loss: 1.2840, Disc Loss: 1.1147, Time: 8.24 sec


Epoch 11, Gen Loss: 1.2301, Disc Loss: 1.1135, Time: 8.56 sec
Epoch 12, Gen Loss: 1.3089, Disc Loss: 1.1302, Time: 11.44 sec
Epoch 13, Gen Loss: 1.2891, Disc Loss: 1.0748, Time: 8.29 sec


2024-08-21 15:09:23.904461: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 14, Gen Loss: 1.3033, Disc Loss: 1.0976, Time: 8.35 sec
Epoch 15, Gen Loss: 1.3096, Disc Loss: 1.0893, Time: 8.85 sec
Epoch 16, Gen Loss: 1.3835, Disc Loss: 1.0895, Time: 8.61 sec
Epoch 17, Gen Loss: 1.3289, Disc Loss: 1.0668, Time: 8.27 sec
Epoch 18, Gen Loss: 1.3865, Disc Loss: 1.0539, Time: 8.40 sec
Epoch 19, Gen Loss: 1.2637, Disc Loss: 1.1221, Time: 11.67 sec
Epoch 20, Gen Loss: 1.4026, Disc Loss: 0.9913, Time: 8.40 sec


Epoch 21, Gen Loss: 1.3136, Disc Loss: 1.1300, Time: 8.31 sec
Epoch 22, Gen Loss: 1.3746, Disc Loss: 1.1071, Time: 8.43 sec
Epoch 23, Gen Loss: 1.3761, Disc Loss: 1.0331, Time: 11.50 sec
Epoch 24, Gen Loss: 1.3092, Disc Loss: 1.0647, Time: 8.63 sec
Epoch 25, Gen Loss: 1.2727, Disc Loss: 1.0866, Time: 8.58 sec
Epoch 26, Gen Loss: 1.2561, Disc Loss: 1.1121, Time: 10.90 sec
Epoch 27, Gen Loss: 1.2184, Disc Loss: 1.0921, Time: 8.89 sec
Epoch 28, Gen Loss: 1.2770, Disc Loss: 1.1291, Time: 11.13 sec
Epoch 29, Gen Loss: 1.3765, Disc Loss: 1.0387, Time: 9.96 sec
Epoch 30, Gen Loss: 1.3557, Disc Loss: 1.0690, Time: 10.96 sec


Epoch 31, Gen Loss: 1.3531, Disc Loss: 1.0996, Time: 10.48 sec
Epoch 32, Gen Loss: 1.4064, Disc Loss: 1.0290, Time: 10.86 sec
Epoch 33, Gen Loss: 1.3881, Disc Loss: 1.0454, Time: 13.31 sec
Epoch 34, Gen Loss: 1.3089, Disc Loss: 1.0754, Time: 10.87 sec
Epoch 35, Gen Loss: 1.3514, Disc Loss: 1.1148, Time: 10.93 sec
Epoch 36, Gen Loss: 1.3794, Disc Loss: 1.0927, Time: 13.72 sec
Epoch 37, Gen Loss: 1.3134, Disc Loss: 1.0893, Time: 11.25 sec
Epoch 38, Gen Loss: 1.2937, Disc Loss: 1.1133, Time: 13.79 sec
Epoch 39, Gen Loss: 1.3084, Disc Loss: 1.0736, Time: 11.29 sec
Epoch 40, Gen Loss: 1.3994, Disc Loss: 1.0359, Time: 11.22 sec


Epoch 41, Gen Loss: 1.3218, Disc Loss: 1.1148, Time: 13.75 sec
Epoch 42, Gen Loss: 1.2406, Disc Loss: 1.1546, Time: 11.22 sec
Epoch 43, Gen Loss: 1.2436, Disc Loss: 1.0712, Time: 11.22 sec
Epoch 44, Gen Loss: 1.2723, Disc Loss: 1.1306, Time: 13.52 sec
Epoch 45, Gen Loss: 1.2347, Disc Loss: 1.0865, Time: 8.52 sec
Epoch 46, Gen Loss: 1.2813, Disc Loss: 1.0834, Time: 9.79 sec
Epoch 47, Gen Loss: 1.2717, Disc Loss: 1.1633, Time: 13.30 sec
Epoch 48, Gen Loss: 1.2553, Disc Loss: 1.0961, Time: 8.58 sec
Epoch 49, Gen Loss: 1.2308, Disc Loss: 1.1184, Time: 8.57 sec
Epoch 50, Gen Loss: 1.3509, Disc Loss: 1.0760, Time: 8.57 sec


In [19]:

def train(dataset, epochs, save_every):
    history = []
    
    # 테스트용 입력 생성
    num_examples_to_generate = 16
    test_noise = tf.random.normal([num_examples_to_generate, 100])
    test_labels = tf.range(0, 10, delta=1)
    test_labels = tf.concat([test_labels, tf.random.uniform([6], minval=0, maxval=10, dtype=tf.int32)], axis=0)
    test_labels = tf.one_hot(test_labels, depth=10)
    
    for epoch in range(epochs):
        start = time.time()
        
        epoch_gen_loss = []
        epoch_disc_loss = []

        for image_batch, label_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch, label_batch)
            epoch_gen_loss.append(gen_loss)
            epoch_disc_loss.append(disc_loss)

        # 에포크 평균 손실 계산
        avg_gen_loss = tf.reduce_mean(epoch_gen_loss)
        avg_disc_loss = tf.reduce_mean(epoch_disc_loss)
        
        history.append({
            'epoch': epoch + 1,
            'gen_loss': avg_gen_loss.numpy(),
            'disc_loss': avg_disc_loss.numpy(),
            'time': time.time() - start
        })

        print(f'Epoch {epoch + 1}, Gen Loss: {avg_gen_loss:.4f}, Disc Loss: {avg_disc_loss:.4f}, Time: {time.time()-start:.2f} sec')

        # 에포크마다 이미지 생성 및 저장
        generate_and_save_images(generator, epoch + 1, test_noise, test_labels)
        
        if (epoch + 1) % save_every == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)
    
    # 학습 히스토리 저장
    np.save(os.path.join(SAVE_DIR, 'training_history.npy'), history)
    
    # 학습 히스토리 그래프 생성 및 저장
    plot_history(history)

def plot_history(history):
    gen_loss = [h['gen_loss'] for h in history]
    disc_loss = [h['disc_loss'] for h in history]
    epochs = [h['epoch'] for h in history]

    plt.figure(figsize=(10, 5))
    plt.plot(epochs, gen_loss, label='Generator Loss')
    plt.plot(epochs, disc_loss, label='Discriminator Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training History')
    plt.savefig(os.path.join(SAVE_DIR, 'training_history.png'))
    plt.close()


# 학습된 모델을 사용하여 특정 클래스의 이미지 생성
def generate_specific_class(class_num):
    noise = tf.random.normal([1, 100])
    one_hot_label = tf.one_hot([class_num], depth=10)
    generated_image = generator([noise, one_hot_label], training=False)
    
    plt.figure(figsize=(4, 4))
    plt.imshow((generated_image[0, :, :, :] + 1) / 2.0)
    plt.axis('off')
    plt.title(f'Generated image for class {class_num}')
    plt.savefig(os.path.join(SAVE_DIR, f'generated_class_{class_num}.png'))
    plt.close()



In [20]:

# 모델 학습
EPOCHS = 50
save_every = 5

train(train_dataset, EPOCHS, save_every)

# 클래스 0부터 9까지의 이미지 생성
for i in range(10):
    generate_specific_class(i)

# airplane
# automobile
# bird
# cat
# deer
# dog
# frog
# horse
# ship
# truck

Epoch 1, Gen Loss: 1.2391, Disc Loss: 1.1336, Time: 14.45 sec
Epoch 2, Gen Loss: 1.2231, Disc Loss: 1.1224, Time: 10.66 sec
Epoch 3, Gen Loss: 1.1666, Disc Loss: 1.1918, Time: 8.71 sec
Epoch 4, Gen Loss: 1.2471, Disc Loss: 1.1846, Time: 11.52 sec
Epoch 5, Gen Loss: 1.3173, Disc Loss: 1.0523, Time: 8.39 sec
Epoch 6, Gen Loss: 1.2188, Disc Loss: 1.1916, Time: 8.28 sec
Epoch 7, Gen Loss: 1.2135, Disc Loss: 1.1196, Time: 8.48 sec
Epoch 8, Gen Loss: 1.1324, Disc Loss: 1.1801, Time: 11.63 sec
Epoch 9, Gen Loss: 1.1063, Disc Loss: 1.1665, Time: 8.31 sec
Epoch 10, Gen Loss: 1.1406, Disc Loss: 1.1790, Time: 8.33 sec
Epoch 11, Gen Loss: 1.1977, Disc Loss: 1.1621, Time: 11.41 sec
Epoch 12, Gen Loss: 1.2465, Disc Loss: 1.1427, Time: 8.59 sec
Epoch 13, Gen Loss: 1.2387, Disc Loss: 1.1574, Time: 10.02 sec
Epoch 14, Gen Loss: 1.2344, Disc Loss: 1.1300, Time: 9.53 sec
Epoch 15, Gen Loss: 1.1864, Disc Loss: 1.1443, Time: 11.53 sec
Epoch 16, Gen Loss: 1.2076, Disc Loss: 1.1256, Time: 8.26 sec
Epoch 17, 

2024-08-21 15:26:27.790893: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 25, Gen Loss: 1.2128, Disc Loss: 1.1564, Time: 12.29 sec
Epoch 26, Gen Loss: 1.1681, Disc Loss: 1.1827, Time: 8.35 sec
Epoch 27, Gen Loss: 1.1880, Disc Loss: 1.1698, Time: 8.15 sec
Epoch 28, Gen Loss: 1.1512, Disc Loss: 1.1361, Time: 8.55 sec
Epoch 29, Gen Loss: 1.1389, Disc Loss: 1.1900, Time: 11.63 sec
Epoch 30, Gen Loss: 1.2089, Disc Loss: 1.1074, Time: 8.37 sec
Epoch 31, Gen Loss: 1.1540, Disc Loss: 1.1718, Time: 8.34 sec
Epoch 32, Gen Loss: 1.1229, Disc Loss: 1.1941, Time: 8.55 sec
Epoch 33, Gen Loss: 1.1657, Disc Loss: 1.1366, Time: 11.78 sec
Epoch 34, Gen Loss: 1.1595, Disc Loss: 1.1547, Time: 8.36 sec
Epoch 35, Gen Loss: 1.2051, Disc Loss: 1.1092, Time: 8.33 sec
Epoch 36, Gen Loss: 1.1469, Disc Loss: 1.1718, Time: 11.51 sec
Epoch 37, Gen Loss: 1.2249, Disc Loss: 1.0984, Time: 9.93 sec
Epoch 38, Gen Loss: 1.2166, Disc Loss: 1.2347, Time: 8.25 sec
Epoch 39, Gen Loss: 1.1846, Disc Loss: 1.2031, Time: 9.16 sec
Epoch 40, Gen Loss: 1.2281, Disc Loss: 1.1205, Time: 11.83 sec
Epo